In [1]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications.vgg16 import VGG16
from PIL import Image
from tensorflow.keras import Model
from tensorflow.python.keras.engine import data_adapter
from tensorflow.python.eager import backprop
from tensorflow.python.util import compat
from tensorflow.keras.models import load_model
from tensorflow.keras import datasets, layers, models

import tensair_aux as tensair
from ResNet50_tensorflow import ResNet

## Define TensorFlow Model

One may create a TensorFlow model as usual but extending TensAIR instead of Model.

In [2]:
class RESNET50(tensair.TensAIR):
    def __init__(self):
        super(RESNET50, self).__init__()
        self.l1 = ResNet()

    @tf.function
    def call(self, x):
        x = self.l1(x)
        return x

In [3]:
#compile TensorFlow model as usual
loss = 'sparse_categorical_crossentropy'
optimizer = 'sgd'
metrics = ['accuracy']

model = RESNET50()
model.compile(
    loss=loss, 
    optimizer=optimizer,   
    metrics=metrics   
)

# Prepare model for deployment

To prepare the model for deployment, we initialize the TF graph making a test prediction.

At last, we initialize the gradients that will be broadcasted via TensAIR (mandatory)

In [4]:
x = tf.zeros((1, 64, 64, 3), dtype=float)

model.predict(x)

2024-03-18 16:14:20.714358: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 1s 998ms/step


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [5]:
model.init_delta(model.trainable_weights)

In [6]:
model.summary()

Model: "resnet50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 res_net (ResNet)            multiple                  23970952  
                                                                 
Total params: 47,888,786
Trainable params: 23,917,832
Non-trainable params: 23,970,954
_________________________________________________________________


# Save Model

To save the model, we simply define the dimentions and types of the input tensors used during training, a pass those (along with the model) to the tensair.define_signatures function) to obtain the signature of the functions

At last, we save the model on the desired location uwing the signatures previously obtained

In [7]:
input_tensors_dims = [[None,64,64,3],[None]]
input_tensors_types = [tf.float32,tf.int32]
input_tensors_structure = (1,1)

In [8]:
signatures = tensair.define_signatures(model, input_tensors_dims,input_tensors_types,input_tensors_structure)

In [9]:
tensair_path = os.environ.get("TENSAIR_PATH")

In [10]:
model.save(tensair_path+"/data/resnet50/resnet50_model.tf", save_format="tf", signatures=signatures)

INFO:tensorflow:Assets written to: /Users/mauro.dalleluccatosi/Documents/GitHub-personal/TensAIR/data/resnet50/resnet50_model.tf/assets


INFO:tensorflow:Assets written to: /Users/mauro.dalleluccatosi/Documents/GitHub-personal/TensAIR/data/resnet50/resnet50_model.tf/assets


# Gradients_size

In [11]:
len(model.delta)

161

In [31]:
weights = 0

for layers in model.delta:    
    number_weights = 1
    for i in layers.shape:
        number_weights  *= i
    weights += number_weights
    
weights

23917832

In [33]:
print('delta_size: ', (weights*4) + 4 + 4 + 100)

delta_size:  95671436


In [34]:
mini_batch_size = 128

In [35]:
print('message_size: ', (((mini_batch_size * 64*64*3) + mini_batch_size) * 4) + 4 + 4 + 8 + 8)

message_size:  6291992
